**QLP Book ch02**

In [1]:
import QuantLib as ql ; import pandas as pd
import myUtil as mu   ; from myABBR import *
tradeDT  = mu.jDT(2022,8,19) ; mu.setEvDT(tradeDT)
settleDT = calJP.advance(tradeDT, Tp2, ql.Days)

# 1.指標金利オブジェクト
curveHDL = ql.RelinkableYieldTermStructureHandle()  
tbrIX    = ql.Tibor(pdFreqSA, curveHDL)
# 2.ヘルパーとカーブオブジェクト
cHelper, tbParRATE = [], []
for knd, tnr, rt in \
        [('depo','6m',0.13636), ('swap','1y',0.15249), ('swap','18m',0.18742),
         ('swap','2y',0.20541), ('swap','3y',0.23156), ('swap','4y',0.25653),
         ('swap','5y',0.28528), ('swap','6y',0.32341), ('swap','7y',0.36591)]:
    if knd == 'depo': cHelper.append(ql.DepositRateHelper(mu.sqHDL(rt/100),tbrIX)) 
    if knd == 'swap': cHelper.append(ql.SwapRateHelper(   mu.sqHDL(rt/100), 
                             ql.Period(tnr), calJP, freqSA, mFLLW, dcA365, tbrIX))
    tbParRATE.append(rt/100)                                   # パーレート用リスト
    
curveOBJ = ql.PiecewiseLogLinearDiscount(Tp2, calJP, cHelper, dcA365)
curveHDL.linkTo(curveOBJ) ; curveOBJ.enableExtrapolation()
# checking
print('tradeDT:{}, referenceDate:{}'
                .format(tradeDT.ISO(),curveOBJ.referenceDate().ISO()))
pd.DataFrame(curveOBJ.nodes(), columns=['node','DF'])[:6]

tradeDT:2022-08-19, referenceDate:2022-08-23


,node,DF
0,"August 23rd, 2022",1.000000
1,"February 24th, 2023",0.999309
2,"August 23rd, 2023",0.998477
3,"February 26th, 2024",0.997170
4,"August 23rd, 2024",0.995896
5,"August 25th, 2025",0.993059


In [2]:
effDT, matDT = mu.jDT(2022,8,23), mu.jDT(2024,8,23)

fixSCD2 = ql.MakeSchedule(effDT, matDT, pdFreqSA, 
                calendar=calJP, rule=dtGENb, endOfMonth=EoMf, 
                convention=mFLLW, terminalDateConvention=mFLLW)
fixSCD2.dates()

(Date(23,8,2022),
 Date(24,2,2023),
 Date(23,8,2023),
 Date(26,2,2024),
 Date(23,8,2024))

In [3]:
[(dt, vl) for dt, vl in zip(tbrIX.timeSeries().dates(), tbrIX.timeSeries().values())]

[]

In [4]:
# タプル -> データフレーム
display( pd.DataFrame( 
                    ([1, 2, 3],
                     [4, 5, 6]) , columns=['A', 'B', 'C']) )
# 辞書型 -> データフレーム
display( pd.DataFrame(
                ({'A':1, 'B':2,'C':3},
                 {'A':4, 'B':5,'C':6}) ,    index=[0, 1] ) )

,A,B,C
0,1,2,3
1,4,5,6


,A,B,C
0,1,2,3
1,4,5,6


- makeTiborCurve() 

In [5]:
# TIBORカーブ
def makeTiborCurve(crvDATA):
    '''makeTiborCurve(crvDATA)->[tbrIX,tbCrvOBJ,tbCrvHDL,tbParRATE]'''
  # 1.指標金利オブジェクト
    tbCrvHDL = ql.RelinkableYieldTermStructureHandle() 
    tbrIX    = ql.Tibor(pdFreqSA, tbCrvHDL)
  # 2. HelperとTONAカーブオブジェクト
    cHelper, tbParRATE = [], []
    for knd, tnr, rt in crvDATA:
       if knd == 'depo': cHelper.append(ql.DepositRateHelper(mu.sqHDL(rt/100),tbrIX)) 
       if knd == 'swap': cHelper.append(ql.SwapRateHelper(   mu.sqHDL(rt/100),
                                ql.Period(tnr), calJP, freqSA, mFLLW, dcA365, tbrIX))
       tbParRATE.append(rt/100)                            # パーレート用リスト
    tbCrvOBJ = ql.PiecewiseLogLinearDiscount(Tp2, calJP, cHelper, dcA365)
    tbCrvHDL.linkTo(tbCrvOBJ) ; tbCrvOBJ.enableExtrapolation()
    return [tbrIX, tbCrvOBJ, tbCrvHDL, tbParRATE]  # 4つのオブジェクトを戻す  

- Tiborスワップ

In [6]:
import QuantLib as ql ; import pandas as pd
import myUtil as mu   ; from myABBR import *
tradeDT  = mu.jDT(2022,8,19) ; mu.setEvDT(tradeDT)
settleDT = calJP.advance(tradeDT, Tp2, ql.Days)
# 1.イールドカーブ オブジェクト
crvDATA = [('depo','6m',0.13636), ('swap','1y',0.15249), ('swap','18m',0.18742),
           ('swap','2y',0.20541), ('swap','3y',0.23156), ('swap','4y',0.25653),
           ('swap','5y',0.28528), ('swap','6y',0.32341), ('swap','7y',0.36591)]
tbrIX, tbCrvOBJ, tbCrvHDL, tbParRATE = mu.makeTiborCurve(crvDATA)
print('checking curve DF: {1:.6f} at {0}'.format(*tbCrvOBJ.nodes()[5]))

# 2.スケジュール オブジェクト 
effDT, matDT = mu.jDT(2022,8,23), mu.jDT(2024,8,23)
fixSCD = ql.Schedule(effDT, matDT, pdFreqSA, calJP, mFLLW, mFLLW, dtGENb, EoMf)
fltSCD = ql.Schedule(effDT, matDT, pdFreqSA, calJP, mFLLW, mFLLW, dtGENb, EoMf)

# 3.スワップオブジェクトとエンジン
payRcv,                ntlAMT,     cpnRT,   sprd                 = \
ql.VanillaSwap.Payer, 10_000_000,  0.012 ,   0.0

swapOBJ = ql.VanillaSwap(payRcv, ntlAMT, fixSCD, cpnRT,       dcA365,
                                         fltSCD, tbrIX, sprd, dcA365)
swapENG = ql.DiscountingSwapEngine(tbCrvHDL)
swapOBJ.setPricingEngine(swapENG)

# 計算結果
print('tradeDT:{}, referenceDate:{}'
                .format(tradeDT.ISO(),tbCrvOBJ.referenceDate().ISO()))
pd.DataFrame([  
    ['固定レグ時価'    ,swapOBJ.legNPV(0)],   ['変動レグ時価',swapOBJ.legNPV(1)],
    ['スワップ時価 NPV',swapOBJ.NPV()],       ['フェアレート',swapOBJ.fairRate()],
    ['フェアスプレッド',swapOBJ.fairSpread()]         ], columns=['計算結果', '']) 

checking curve DF: 0.993059 at August 25th, 2025
tradeDT:2022-08-19, referenceDate:2022-08-23


,計算結果,
0,固定レグ時価,-239781.110343
1,変動レグ時価,41044.531563
2,スワップ時価 NPV,-198736.578780
3,フェアレート,0.002054
4,フェアスプレッド,0.009946


- MakeSchedule

In [7]:
fixSCDmk = ql.MakeSchedule(effDT, matDT, pdFreqSA, calendar=calJP)
print(list(fixSCDmk))

[Date(23,8,2022), Date(24,2,2023), Date(23,8,2023), Date(26,2,2024), Date(23,8,2024)]


- 変動レグ キャッシュフロー表

In [8]:
dfSWP = pd.DataFrame({
    'fixingDate': cpn.fixingDate().ISO(),
    'accruStart': cpn.accrualStartDate().ISO(),
    'accruEnd':   cpn.accrualEndDate().ISO(),
    'payDate':    cpn.date(),                             # Date class(No ISO form)
    'days':       dcA365.dayCount(cpn.accrualStartDate(),cpn.accrualEndDate()),
    'rate':       cpn.rate(),
    'spread':     cpn.spread(),
    'amount':     cpn.amount(),
    } for cpn in map(ql.as_floating_rate_coupon, swapOBJ.leg(1)))       #変動レグ=1
# ディスカウントファクター(DF)
psDF = [1.0                   for dt in dfSWP.payDate if dt <= settleDT] #past   DF
fuDF = [tbCrvOBJ.discount(dt) for dt in dfSWP.payDate if settleDT < dt ] #future DF
dfSWP = pd.concat([dfSWP, pd.DataFrame(psDF+fuDF, columns=['DF']) ], axis=1)
dfSWP.payDate = dfSWP.payDate.map(lambda x: x.ISO())                         # ISOへ

dfSWP.style.format({'rate':'{:.6%}','spread':'{:.2%}','amount':'{:,.2f}','DF':'{:.8f}'})

,fixingDate,accruStart,accruEnd,payDate,days,rate,spread,amount,DF
0,2022-08-19,2022-08-23,2023-02-24,2023-02-24,185,0.136360%,0.00%,"6,911.40",0.99930934
1,2023-02-21,2023-02-24,2023-08-23,2023-08-23,180,0.169082%,0.00%,"8,338.28",0.99847678
2,2023-08-21,2023-08-23,2024-02-26,2024-02-26,187,0.255717%,0.00%,"13,101.12",0.99717037
3,2024-02-21,2024-02-26,2024-08-23,2024-08-23,179,0.261022%,0.00%,"12,800.82",0.99589555


- イテレータ/ジェネレータ

In [9]:
df1 = pd.DataFrame([{'A':1, 'B':1*2}, {'A':2, 'B':2*2}, {'A':3, 'B':3*2}])
df2 = pd.DataFrame({'A':x,  'B':x*2} for x in range(1,4))
print('df1 \n', df1) ; print('df2 \n', df2)

df1 
    A  B
0  1  2
1  2  4
2  3  6
df2 
    A  B
0  1  2
1  2  4
2  3  6


- Swapキャッシュフロー表

In [10]:
def swapCashFlow(swapOBJ, curveOBJ, leg=1, dc=dcA365): 
    '''swapCashFlow(swapOBJ, curveOBJ, leg=1:FLoat  0:Fix)-> DataFrame)'''  
    if leg == 1:  # 変動ﾚｸﾞ leg(1)
        dfSWP = pd.DataFrame({
            'fixingDate': cpn.fixingDate().ISO(),
            'accruStart': cpn.accrualStartDate().ISO(),
            'accruEnd':   cpn.accrualEndDate().ISO(),
            'payDate':    cpn.date(),                            # No ISO form
            'days':       dc.dayCount(cpn.accrualStartDate(),cpn.accrualEndDate()),
            'rate':       cpn.rate(),
            'spread':     cpn.spread(),
            'amount':     cpn.amount(),
            } for cpn in map(ql.as_floating_rate_coupon, swapOBJ.leg(1)))
    else:          # 固定ﾚｸﾞ leg(0)
        dfSWP = pd.DataFrame({
            'nominal':    cpn.nominal(),
            'accruStart': cpn.accrualStartDate().ISO(),
            'accruEnd':   cpn.accrualEndDate().ISO(),
            'payDate':    cpn.date(),
            'days':       dc.dayCount(cpn.accrualStartDate(),cpn.accrualEndDate()),
            'rate':       cpn.rate(),
            'amount':     cpn.amount()
            } for cpn in map(ql.as_fixed_rate_coupon, swapOBJ.leg(0)))
    # ディスカウントファクター(DF)
    settleDT = curveOBJ.referenceDate()
    psDF = [1.0                   for dt in dfSWP.payDate if dt <= settleDT] # past   DF
    fuDF = [curveOBJ.discount(dt) for dt in dfSWP.payDate if settleDT < dt ] # future DF
    dfSWP = pd.concat([dfSWP, pd.DataFrame(psDF+fuDF, columns=['DF']) ], axis=1)
    dfSWP.payDate = dfSWP.payDate.map(lambda x: x.ISO())                        # ISOへ
    return dfSWP

**Fixing**

In [11]:
tbrIX.addFixing(mu.jDT(2022,8,19),0.13636/100,True)

In [12]:
# 新しい日付設定
tradeDT  = mu.jDT(2023,5,1) ; mu.setEvDT(tradeDT)
settleDT = calJP.advance(tradeDT, Tp2, ql.Days)

# # fixing dateとfixing例
prevPayDT = settleDT if swapOBJ.floatingSchedule().startDate() >= settleDT \
                   else swapOBJ.floatingSchedule().previousDate(settleDT)
fixedDT = calJP.advance(prevPayDT, -ql.Period(str(Tp2)+'d'))
tbrIX.addFixing(fixedDT,0.1/100,True)

# float leg CF (leg=1)
dfFixCF = mu.swapCashFlow(swapOBJ, tbCrvOBJ, leg=1)
dfFixCF.style.format({'rate':'{:.6%}','spread':'{:.2%}','amount':'{:,.2f}','DF':'{:.8f}'})

,fixingDate,accruStart,accruEnd,payDate,days,rate,spread,amount,DF
0,2022-08-19,2022-08-23,2023-02-24,2023-02-24,185,0.136360%,0.00%,"6,911.40",1.00000000
1,2023-02-21,2023-02-24,2023-08-23,2023-08-23,180,0.100000%,0.00%,"4,931.51",0.99960048
2,2023-08-21,2023-08-23,2024-02-26,2024-02-26,187,0.155450%,0.00%,"7,964.13",0.99880502
3,2024-02-21,2024-02-26,2024-08-23,2024-08-23,179,0.221531%,0.00%,"10,864.14",0.99772108


In [13]:
[(dt, va) for dt, va in zip(tbrIX.timeSeries().dates(), 
                            tbrIX.timeSeries().values())]

[(Date(19,8,2022), 0.0013636000000000002), (Date(21,2,2023), 0.001)]

- 固定レグ

In [14]:
# 日付を元へ戻す
tradeDT  = mu.jDT(2022,8,19) ; mu.setEvDT(tradeDT)
settleDT = calJP.advance(tradeDT, Tp2, ql.Days)

In [15]:
# fixed leg CF (leg=0)
dfFixCF = mu.swapCashFlow(swapOBJ, tbCrvOBJ, leg=0)
display( dfFixCF.style.format(
    {'nominal':'{:,.2f}', 'rate':'{:.6%}', 'amount':'{:,.2f}','DF':'{:.8f}'}) )
print('アニュイティ:', (dfFixCF.days/365 * dfFixCF.DF).sum() )

,nominal,accruStart,accruEnd,payDate,days,rate,amount,DF
0,"10,000,000.00",2022-08-23,2023-02-24,2023-02-24,185,1.200000%,"60,821.92",0.99930934
1,"10,000,000.00",2023-02-24,2023-08-23,2023-08-23,180,1.200000%,"59,178.08",0.99847678
2,"10,000,000.00",2023-08-23,2024-02-26,2024-02-26,187,1.200000%,"61,479.45",0.99717037
3,"10,000,000.00",2024-02-26,2024-08-23,2024-08-23,179,1.200000%,"58,849.32",0.99589555


アニュイティ: 1.998175919523169


**7. フォワードスワップレート**

In [16]:
import numpy as np ; import myUtil as mu ; 
# フォワードスワップのスケジュールオブジェクト
effDTf, matDTf = mu.jDT(2023,8,23), mu.jDT(2025,8,23) 
fixSCDf = ql.Schedule(effDTf, matDTf, pdFreqSA, calJP, mFLLW, mFLLW, dtGENb, EoMf)

# アニュイティ計算
discFCT = np.array([tbCrvOBJ.discount(d) for d in fixSCDf][1:]) 
tnrLST = np.diff([dcA365.yearFraction(fixSCDf.startDate(), d) for d in fixSCDf]) 
annuity = np.sum(tnrLST * discFCT)

print('discount factors: ', discFCT)
print('tenor list      : ', tnrLST)
print('アニュイティ     : ', annuity)

discount factors:  [0.99717037 0.99589555 0.99445706 0.99305925]
tenor list      :  [0.51232877 0.49041096 0.50958904 0.49589041]
アニュイティ     :  1.998490138388628


In [17]:
# 3.スケジュールオブジェクト(フォワードスワップ) 
effDTf, matDTf = mu.jDT(2023,8,23), mu.jDT(2025,8,23) 
fixSCDf = ql.Schedule(effDTf, matDTf, pdFreqSA, calJP, mFLLW, mFLLW, dtGENb, EoMf)
fltSCDf = ql.Schedule(effDTf, matDTf, pdFreqSA, calJP, mFLLW, mFLLW, dtGENb, EoMf)

# 4.スワップオブジェクトの作成とエンジンの設定
swapOBJf = ql.VanillaSwap(ql.VanillaSwap.Payer, ntlAMT, fixSCDf,      cpnRT, dcA365,
                                                       fltSCDf, tbrIX, sprd, dcA365)
swapENGf = ql.DiscountingSwapEngine(tbCrvHDL)
swapOBJf.setPricingEngine(swapENGf)

# 5. 計算結果
print('Fixed(leg0) NPV : {:,.2f}'.format(swapOBJf.legNPV(0)))
print('Float(leg1) NPV : {:,.2f}'.format(swapOBJf.legNPV(1)))
print('Swap NPV        : {:,.2f}'.format(swapOBJf.NPV()))
print('fairRate        : {: .6%}'.format(swapOBJf.fairRate()))
print('fairSpread      : {: .6%}'.format(swapOBJf.fairSpread()))

Fixed(leg0) NPV : -239,818.82
Float(leg1) NPV : 54,175.33
Swap NPV        : -185,643.49
fairRate        :  0.271081%
fairSpread      :  0.928919%


In [18]:
# 固定ﾚｸﾞ leg(0)
cf0 = pd.DataFrame({
    'nominal':    cf.nominal(),
    'accruStart': cf.accrualStartDate().ISO(),
    'accruEnd':   cf.accrualEndDate().ISO(),
    'payDate':    cf.date().ISO(),
    'days':       cf.accrualEndDate() - cf.accrualStartDate(),
    'rate':       cf.rate(),
    'amount':     cf.amount(),
    'DF':         tbCrvOBJ.discount(cf.date())
    } for cf in map(ql.as_coupon, swapOBJf.leg(0)))

display( cf0.style.format({'nominal':'{:,.2f}', 'days':'{:.0f}', 'rate':'{:.5%}', 
                  'amount':'{:,.2f}', 'DF':'{:.8f}' }) )
print('アニュイティ:', (cf0.days/365 * cf0.DF).sum() )

,nominal,accruStart,accruEnd,payDate,days,rate,amount,DF
0,"10,000,000.00",2023-08-23,2024-02-26,2024-02-26,187,1.20000%,"61,479.45",0.99717037
1,"10,000,000.00",2024-02-26,2024-08-23,2024-08-23,179,1.20000%,"58,849.32",0.99589555
2,"10,000,000.00",2024-08-23,2025-02-25,2025-02-25,186,1.20000%,"61,150.68",0.99445706
3,"10,000,000.00",2025-02-25,2025-08-25,2025-08-25,181,1.20000%,"59,506.85",0.99305925


アニュイティ: 1.9984901383886282
